![](https://drive.google.com/uc?export=view&id=1-5X9OUkA-C2Ih1gOS9Jd7GmkTWUEpDg1)

# Laboratorio 05: Clasificación lineal y no lineal

## Introducción a Data Science
   
<center>
    <img src='images/national_language.jpg'style="width: 300px;">
</center>

**Profesor**: Juan Bekios Calfa

**Carreras**: ICCI, IECI e IenCI


# Descripción del problema

Se desea construir un clasificador que permita predecir la emoción con que fue escrito un texto. 

Para esto se contará con una base de datos de películas con sus respectivas revisiones. 

Cada texto contiene una etiqueta que la define como una revisión positiva o negativa.

# Base de datos

*  El conjunto de datos que vamos a utilizar para este laboratorio puede ser descargado de _Cornell Natural Language Processing Group_. 
*  El conjunto de datos consiste en un total de **2000 documentos**. La mitad de los documentos contienen **reseñas positivas** sobre una película, mientras que la otra mitad contiene **reseñas negativas**. 
*  Más detalles sobre el conjunto de datos se pueden encontrar en este enlace: http://www.cs.cornell.edu/people/pabo/movie-review-data/poldata.README.2.0.txt.

# Análisis de sentimientos con Scikit-Learn

Una vez descargados los datos, debemos realizar un conjunto de pasos para poder construir el modelo de manera adecuada:

1. Importar las liberías.
2. Importar la base de datos.
3. Acondicionar el texto.
4. Convertir el texto a un formato que pueda aprender el modelo.
5. Entrenar el modelo.
6. Evaluar el modelo.
7. Guardar el modelo.

## 1. Importar las librerías



In [ ]:
import numpy as np # Librería para trabajar con matrices
import re          # Librería de expresiones regulares
import nltk        # Librería para procesar texto
from sklearn.datasets import load_files # Libreria para cargar los textos de un directorio
nltk.download('stopwords')              # Descargar palabras no deseadas
nltk.download('wordnet')                # Bajar diccionario wordnet
import pickle                           # Librería para serializar objetos
from nltk.corpus import stopwords       # Módulo para trabajar con stopwords

## 2. Importando la base de datos

* Utilizaremos la función _load_files_ de la biblioteca sklearn_datasets para importar el conjunto de datos a nuestra aplicación. 
* La función _load_files_ divide automáticamente el conjunto de datos en datos de entrada y etiquetas. 
* Por ejemplo, colocamos la ruta  **"txt_sentoken"**. La función _load_files_ tratará cada carpeta dentro de la carpeta "txt_sentoken" como una categoría y a todos los documentos dentro de esa carpeta se les asignará su categoría correspondiente.
* El resultado se entrega como una lista: Una contiene los textos de la revisión y otra la etiqueta (1-Neg y 0-Pos)

In [ ]:
movie_data = load_files(r"txt_sentoken")
X, y = movie_data.data, movie_data.target

print(X[0])

## 3. Acondicionar el texto

* Una vez que se ha importado el conjunto de datos, el siguiente paso es preprocesar el texto. 
* El texto puede contener números, caracteres especiales y espacios no deseados. Dependiendo del problema que enfrentemos, es posible que necesitemos o no eliminar estos caracteres especiales y números del texto. 
* En el laboratorio **eliminaremos todos los caracteres especiales, números y espacios no deseados** del texto. 

In [ ]:
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remover todos los caracteres especiales
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # Remover todos los caracteres individuales
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remover todos las caracteres individuales que están en el inicio
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Sustituir multiples espacios por uno solo
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Remover prefijp 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Convertir todo a minúsculas
    document = document.lower()
    
    # Lemmatización
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

## 4. Convertir el texto a un formato que pueda aprender el modelo.

Las máquinas, a diferencia de los humanos, no pueden comprender el **texto en bruto**. Las máquinas solo pueden ver **números**. En particular, las técnicas estadísticas como el aprendizaje automático solo pueden tratar con números. Por lo tanto, necesitamos convertir nuestro texto en números.

Existen dos aproximaciones para convertir el texto en una estructura entendible para los algoritmos: 
* _The bag of word model_
* _Word embedding model_


### 4.1 _Bag of Words_

El modelo "bolsa de palabras" (del inglés, _Bag of Words_) es un método que se utiliza en el procesado del lenguaje para representar documentos ignorando el orden de las palabras. 

En este modelo, cada documento parece una bolsa que contiene algunas palabras. Por lo tanto, este método permite un modelado de las palabras basado en **diccionarios**, donde cada bolsa contiene unas cuantas palabras del diccionario. 


ref: https://es.wikipedia.org/wiki/Modelo_bolsa_de_palabras

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

# Parámetros
# max_features: Cantidad de features que serán seleccionadas, se escogen las que ocurren más.
# min_df: Cantidad mínima de documentos que contienen la palabra.
# max_df: Debemos incluir solo aquellas palabras que aparecen en un máximo del 70% de todos los documentos.
# stop_words: Diccionario de palabras que son serán consideradas en la vectorización
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

### 4.2 TF-IDF (_Term frequency_ – _Inverse document frequency_)

El valor de **TF-IDF** para una palabra en un documento en particular es mayor si la frecuencia de aparición de esa palabra es mayor en ese documento específico pero menor en todos los demás documentos.

* _**T**erm **F**requency_: $\frac{\#\; de\; ocurrencias\; de\; una\; palabra}{Total\; de\; palabras\; en\; el\; documento}$
* _**I**nverse **D**ocument **F**requency_: $log \left( \frac{Total\; de\; documentos}{\#\; de\; documentos\; que\; contienen\; la\; palabra}\right)$

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [ ]:
También se puede hacer directamente...

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = tfidfconverter.fit_transform(documents).toarray()

## 5.Entrenar el modelo.

### 5.1 Preparar conjunto de entrenamiento y pruebas

Para entrenar primero dividiremos el conjunto de datos en un conjunto de **entrenamiento** y otro de **pruebas**.

* 80% para el conjunto de entrenamiento.
* 20% para el conjunto de pruebas.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### 5.2 Entrenamiento del modelo de clasificación y predicción de sentiemientos

In [ ]:
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

## 6. Evaluar el modelo (I)

Para evaluar el desempeño de un modelo de clasificación como el que acabamos de entrenar, podemos usar métricas como la matriz de confusión, `la medida F1` y la `precisión`.

<center>
    <img src='images/matriz-confusion.png'style="width: 400px;">
</center>

Para encontrar estos valores, podemos usar las utilidades class_report, confusion_matrix y precision_score de la biblioteca `sklearn.metrics`.

## 6. Evaluar el modelo (II)

Para evaluar el desempeño de un modelo de clasificación como el que acabamos de entrenar, podemos usar métricas como la matriz de confusión, `la medida F1` y la `precisión`.

* $Exactitud=\frac{VP+VN}{Total}$
* $\text{Precisión}=\frac{VP}{\text{Total clasificados positivos (VP+FP)}}$



Para encontrar estos valores, podemos usar las utilidades class_report, confusion_matrix y precision_score de la biblioteca `sklearn.metrics`.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

## 7. Guardar el modelo

Podemos guardar el modelo generado como un objeto `pickle` en Python. 

In [ ]:
with open('text_classifier', 'wb') as picklefile:
    pickle.dump(classifier,picklefile)

###  Cargar el modelo

In [ ]:
with open('text_classifier', 'rb') as training_model:
    model = pickle.load(training_model)

### Probar el modelo cargado

In [ ]:
y_pred2 = model.predict(X_test)

print(confusion_matrix(y_test, y_pred2))
print(classification_report(y_test, y_pred2))
print(accuracy_score(y_test, y_pred2)) 

# Entrega de laboratorio

**Fecha**: 30/Dic/2020 hasta las 23:00 horas.

Se pide:
* Evaluar el clasificador lineal `SGDClassifier` con función de pérdida `hinge` y `regresión logística`. Pruebe el clasificador con regularización L1 y L2, y sin regularización. Utilice validación cruzada 5-_fold_ estratificada.
* Evaluar un clasificador lineal SVM con kernel: https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html#sphx-glr-auto-examples-svm-plot-rbf-parameters-py. Pruebe diferentes parámetros como en el ejemplo.
* Compare el rendimiento de ambos algoritmos e indique cual a su criterio es el mejor y el peor.
* Utilice la plantilla de google colab que se encuentra en el campus virtual para realizar el informe.